In [1]:
# including the project directory to the notebook level
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np

In [2]:
#importing modules
from tensorflow.keras import backend as K

from dataprocess import dataprocessor as dp
from nn_source import models as mp

In [3]:
# read the pickled file for ahu data
ahudata = dp.readfile('../data/processed/ahu1energy.pkl')

# return pickled df
ahu = ahudata.return_df(processmethods=['file2df'])

# read the pickled file for ghi data
ghidata = dp.readfile('../data/processed/ghi.pkl')

# return pickled df
ghi = ghidata.return_df(processmethods=['file2df'])

In [4]:
# selecting only required columns and rearranging them
ahu = ahu[[
    'AHU_1 outdoorAirTemp', 'WeatherDataProfile humidity',
    'AHU_1 supplyAirTemp', 'HW_BTU_METER currentKbtuDeltaReading',
    'CHW_BTU_METER currentKbtuDeltaReading'
]]

# renaming columns
ahu.columns = ['oat', 'orh', 'sat', 'hwe', 'cwe']

# Total energy is sum of heating and coling
ahu['totale'] = ahu['hwe'] + ahu['cwe']
# dropping heating and cooling energy columns
ahu = ahu.drop(columns=['hwe', 'cwe'])

In [5]:
# merging ahu and ghi data
df = dp.merge_df_columns([ahu, ghi])

# rearranging columns
df = df[['oat', 'orh', 'sat', 'Ghi', 'totale']]

In [6]:
df.head()

,oat,orh,sat,Ghi,totale
2018-07-12 13:55:00,89.554909,76.0,75.443275,872.0,35.590466
2018-07-12 14:00:00,89.863190,76.0,75.443275,866.0,33.027813
2018-07-12 14:05:00,89.863190,76.0,75.443275,859.0,31.769627
2018-07-12 14:10:00,90.484146,69.0,75.443275,848.0,34.195641
2018-07-12 14:15:00,90.826981,69.0,75.443275,836.0,31.082222


In [7]:
df['HourSine'] = np.sin(2*np.pi*df.index.hour/24.0)
df['HourCos'] = np.cos(2*np.pi*df.index.hour/24.0)
df['WeekSine'] = np.sin(2*np.pi*df.index.week/52.0)
df['WeekCos'] = np.cos(2*np.pi*df.index.week/52.0)
df['DayofWeekSine'] = np.sin(2*np.pi*df.index.dayofweek/7.0)
df['DayofWeekCos'] = np.cos(2*np.pi*df.index.dayofweek/7.0)

In [8]:
df = df[[
    'oat', 'orh', 'sat', 'Ghi', 'HourSine', 'HourCos', 'WeekSine', 'WeekCos',
    'DayofWeekSine', 'DayofWeekCos', 'totale'
]]

In [9]:
df.head()

,oat,orh,sat,Ghi,HourSine,HourCos,WeekSine,WeekCos,DayofWeekSine,DayofWeekCos,totale
2018-07-12 13:55:00,89.554909,76.0,75.443275,872.0,-0.258819,-0.965926,-0.239316,-0.970942,0.433884,-0.900969,35.590466
2018-07-12 14:00:00,89.863190,76.0,75.443275,866.0,-0.500000,-0.866025,-0.239316,-0.970942,0.433884,-0.900969,33.027813
2018-07-12 14:05:00,89.863190,76.0,75.443275,859.0,-0.500000,-0.866025,-0.239316,-0.970942,0.433884,-0.900969,31.769627
2018-07-12 14:10:00,90.484146,69.0,75.443275,848.0,-0.500000,-0.866025,-0.239316,-0.970942,0.433884,-0.900969,34.195641
2018-07-12 14:15:00,90.826981,69.0,75.443275,836.0,-0.500000,-0.866025,-0.239316,-0.970942,0.433884,-0.900969,31.082222


In [10]:
# Creating a list of 7 day dataframes for training
dflist = dp.df2dflist(df, subsequence=True, period=1, days=7, hours=0)

In [11]:
X_train, X_test, y_train, y_test, X_scaler, y_scaler = dp.df2arrays(
    dflist[1],
    predictorcols=['oat', 'orh', 'sat', 'Ghi',],
    outputcols=['totale'],
    scaling=True,
    reshaping=True,
    lag=-1)

In [12]:
for i in [X_train, X_test, y_train, y_test]:
    print(i.shape)

(1511, 1, 4)
(504, 1, 4)
(1511, 1, 1)
(504, 1, 1)


In [13]:
!rm -rf ../results/lstmtrain2

In [14]:
# create the results directory
try:
    os.mkdir('../results/lstmtrain2/')
except FileExistsError:
    files = os.listdir('../results/lstmtrain2/')
    for f in files:
        os.remove('../results/lstmtrain2/' + f)
        
os.mkdir('../results/lstmtrain2/loginfo')

In [15]:
try:
    del model
except NameError:
    pass

K.clear_session()

In [16]:
#Instantiate learner model
model = mp.lstm_model('../results/lstmtrain2/',
                      inputdim=X_train.shape[-1],
                      outputdim=1,
                      period=1)

# Desing model architecture
model.design_model(lstmhiddenlayers=[16] * 1,
                   densehiddenlayers=[32] * 5,
                   dropoutlist=[[], []],
                   batchnormalizelist=[[], []])

In [17]:
model.show_model()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 1, 4)]            0         
_________________________________________________________________
reshape_layer (Reshape)      (None, 4)                 0         
_________________________________________________________________
input_repeater (RepeatVector (None, 1, 4)              0         
_________________________________________________________________
lstm (LSTM)                  (None, 1, 16)             1344      
_________________________________________________________________
dense (Dense)                (None, 1, 32)             544       
_________________________________________________________________
dense_1 (Dense)              (None, 1, 32)             1056      
_________________________________________________________________
dense_2 (Dense)              (None, 1, 32)             1056  

In [18]:
# creating early sstopping, model checkpointing and learning reate changing callbacks
model.model_callbacks()

In [19]:
model.train_model(X_train, y_train, X_test, y_test, epochs = 50)

Train on 1511 samples, validate on 504 samples
Epoch 1/50
1511/1511 - 4s - loss: 1.1378 - val_loss: 0.7971
Epoch 2/50
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../results/lstmtrain2/LSTM_model_02_0.45/assets
1511/1511 - 6s - loss: 0.6181 - val_loss: 0.4454
Epoch 3/50
1511/1511 - 1s - loss: 0.3421 - val_loss: 0.2373
Epoch 4/50
INFO:tensorflow:Assets written to: ../results/lstmtrain2/LSTM_model_04_0.12/assets
1511/1511 - 6s - loss: 0.1819 - val_loss: 0.1235
Epoch 5/50
1511/1511 - 1s - loss: 0.0976 - val_loss: 0.0670
Epoch 6/50
INFO:tensorflow:Assets written to: ../results/lstmtrain2/LSTM_model_06_0.04/assets
1511/1511 - 6s - loss: 0.0568 - val_loss: 0.0406
Epoch 7/50
1511/1511 - 1s - loss: 0.0369 - val_loss: 0.0273
Epoch 8/50
INFO:tensorflow:Assets written to: ../results/lstmtrain2/LSTM_model_08_0.02/assets
1511/1511 - 5s - loss: 0.0270 - val_loss: 0.0209
Epoch 9/50
1511/1511 - 1s - loss: 0.0219 - val_loss: 0.0181


In [20]:
preds_train, preds_test = model.evaluate_model(X_train,
                                                         y_train,
                                                         X_test,
                                                         y_test,
                                                         y_scaler,
                                                         saveplot=True)

In [ ]:
dp.dataframeplot(dflist[2], lazy=False, style='-', legend=True)